<a href="https://colab.research.google.com/github/Duyhoan99/Smiley-Hit/blob/main/R%E1%BB%91i_lo%E1%BA%A1n_gi%E1%BA%A5c_ng%E1%BB%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas "scikit-learn>=1.3.0" matplotlib seaborn optuna

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
import sys
import joblib

# --- KIỂM TRA THƯ VIỆN ---
try:
    import optuna
except ImportError:
    print("Lỗi: Thiếu thư viện 'optuna'. Vui lòng chạy: pip install optuna")
    sys.exit()

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import (f1_score, accuracy_score, precision_score, recall_score,
                             classification_report, confusion_matrix)

# --- TẮT CÁC CẢNH BÁO ---
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# === HÀM TÍNH METRICS ===
def calculate_metrics_summary(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1-Macro': f1_score(y_true, y_pred, average='macro', zero_division=0),
        'F1-Weighted': f1_score(y_true, y_pred, average='weighted', zero_division=0),
        'Precision-Macro': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'Recall-Macro': recall_score(y_true, y_pred, average='macro', zero_division=0)
    }

# === HÀM EDA ===
def run_full_visual_eda(df, target_col):
    numerical_cols = [
        'Age', 'Sleep Duration', 'Physical Activity Level', 'Stress Level',
        'Heart Rate', 'Daily Steps', 'Systolic_BP', 'Diastolic_BP'
    ]
    categorical_cols = ['Gender', 'Occupation', 'BMI Category']

    numerical_cols = [col for col in numerical_cols if col in df.columns]
    categorical_cols = [col for col in categorical_cols if col in df.columns]

    try:
        # 1. Phân phối Biến mục tiêu
        plt.figure(figsize=(8, 6))
        ax = sns.countplot(data=df, x=target_col, palette='viridis',
                      order=df[target_col].value_counts().index)
        plt.title(f'Phân phối Biến mục tiêu ({target_col})', fontsize=14, pad=15)
        plt.xlabel('Rối loạn giấc ngủ', fontsize=11)
        plt.ylabel('Số lượng', fontsize=11)
        for container in ax.containers:
            ax.bar_label(container)
        plt.tight_layout()
        plt.savefig('eda_target_distribution.png', dpi=150, bbox_inches='tight')
        plt.close()

        # 2. EDA - Numerical
        n_nums = len(numerical_cols)
        n_cols_num = 3
        n_rows_num = math.ceil(n_nums / n_cols_num)

        plt.figure(figsize=(7 * n_cols_num, 5 * n_rows_num))
        for i, col in enumerate(numerical_cols):
            plt.subplot(n_rows_num, n_cols_num, i + 1)
            sns.histplot(df[col], kde=True, bins=20, color='royalblue')
            plt.title(f'Phân phối của {col}', fontsize=12)
            plt.xlabel(col, fontsize=10); plt.ylabel('Tần suất', fontsize=10)
        plt.suptitle('EDA - Phân phối các Đặc trưng Số', fontsize=18, y=1.02)
        plt.tight_layout()
        plt.savefig('eda_numerical_features.png', dpi=150, bbox_inches='tight')
        plt.close()

        # 3. EDA - Feature vs Target
        plt.figure(figsize=(15, 10))
        plt.subplot(2, 2, 1)
        sns.boxplot(data=df, x=target_col, y='Stress Level', palette='Set2')
        plt.title('Mức độ Stress vs. Rối loạn giấc ngủ', fontsize=12)

        plt.subplot(2, 2, 2)
        sns.boxplot(data=df, x=target_col, y='Sleep Duration', palette='Set2')
        plt.title('Thời gian ngủ vs. Rối loạn giấc ngủ', fontsize=12)

        plt.subplot(2, 2, 3)
        sns.countplot(data=df, x='BMI Category', hue=target_col, palette='Set3')
        plt.title('BMI vs. Rối loạn giấc ngủ', fontsize=12)

        plt.subplot(2, 2, 4)
        sns.histplot(data=df, x='Age', hue=target_col, multiple='stack', kde=True)
        plt.title('Phân phối Tuổi theo Rối loạn giấc ngủ', fontsize=12)

        plt.suptitle('EDA - Mối quan hệ giữa Đặc trưng và Mục tiêu', fontsize=18, y=1.02)
        plt.tight_layout()
        plt.savefig('eda_features_vs_target.png', dpi=150, bbox_inches='tight')
        plt.close()

    except Exception as e:
        print(f"[EDA] ✗ Lỗi: {e}")

# === HÀM CHÍNH ===
def run_complete_ml_workflow_SleepDisorder():

    # 1.1. Tải dữ liệu
    try:
        df = pd.read_csv("/content/Sleep_health_and_lifestyle_dataset.csv")
    except FileNotFoundError:
        print("✗ LỖI: Không tìm thấy file 'Sleep_health_and_lifestyle_dataset.csv'")
        return

    # 1.2. Feature Engineering
    df = df.drop('Person ID', axis=1, errors='ignore')
    if 'Blood Pressure' in df.columns:
        df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
        df = df.drop('Blood Pressure', axis=1)

    # 1.3. Chuẩn bị Target
    TARGET = 'Sleep Disorder'
    df[TARGET] = df[TARGET].fillna('None')

    print("\nPhân phối các lớp (Dataset):")
    class_counts = df[TARGET].value_counts()
    for cls, count in class_counts.items():
        pct = count / len(df) * 100
        print(f"  {cls:15} : {count:3} mẫu ({pct:5.1f}%)")

    # 1.4. Chạy EDA (Lưu ảnh ngầm)
    run_full_visual_eda(df.copy(), TARGET)

    # 1.5. Chuẩn bị Features
    LEAK_FEATURE = 'Quality of Sleep'
    FEATURES = [col for col in df.columns if col not in [TARGET, LEAK_FEATURE]]
    X = df[FEATURES]
    y = df[TARGET]

    # 1.6. Tách Dữ liệu
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print("\nPhân phối lớp trong Train set:")
    train_counts = y_train.value_counts()
    for cls, count in train_counts.items():
        print(f"  {cls:15} : {count:3} mẫu")

    # 1.7. Pipeline Tiền xử lý
    numeric_features = [
        'Age', 'Sleep Duration', 'Physical Activity Level', 'Stress Level',
        'Heart Rate', 'Daily Steps', 'Systolic_BP', 'Diastolic_BP'
    ]
    nominal_features = ['Gender', 'Occupation']
    ordinal_features = ['BMI Category']

    numeric_features = [col for col in numeric_features if col in X_train.columns]
    nominal_features = [col for col in nominal_features if col in X_train.columns]
    ordinal_features = [col for col in ordinal_features if col in X_train.columns]

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    nominal_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    bmi_order = ['Normal', 'Normal Weight', 'Overweight', 'Obese']
    detected_bmi = [cat for cat in df['BMI Category'].dropna().unique() if cat in bmi_order]

    if detected_bmi:
        ordinal_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ordinal', OrdinalEncoder(
                categories=[detected_bmi],
                handle_unknown='use_encoded_value',
                unknown_value=-1
            ))
        ])
    else:
        ordinal_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('nom', nominal_transformer, nominal_features),
            ('ord', ordinal_transformer, ordinal_features)
        ],
        remainder='drop',
        sparse_threshold=0,
        verbose_feature_names_out=False
    )

    # --- GIAI ĐOẠN 2: Benchmarking ---
    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scoring = {'f1_macro': 'f1_macro', 'accuracy': 'accuracy'}

    models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=3000, class_weight='balanced'),
        'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced'),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'Support Vector Machine': SVC(random_state=42, class_weight='balanced'),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        'AdaBoost': AdaBoostClassifier(random_state=42, algorithm='SAMME'),
        'Gaussian Naive Bayes': GaussianNB(),
        'MLP Classifier': MLPClassifier(random_state=42, max_iter=1000),
        'Linear Discriminant': LinearDiscriminantAnalysis()
    }

    results_data = []

    for name, model in models.items():
        full_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
        try:
            cv_results = cross_validate(
                full_pipeline, X_train, y_train,
                cv=cv_strategy, scoring=scoring, n_jobs=-1, error_score='raise'
            )
            f1_macro_mean = np.mean(cv_results['test_f1_macro'])
            f1_macro_std = np.std(cv_results['test_f1_macro'])
            accuracy_mean = np.mean(cv_results['test_accuracy'])

            results_data.append({
                "Mô hình": name,
                "F1-Macro": f1_macro_mean,
                "Std": f1_macro_std,
                "Accuracy": accuracy_mean
            })
        except Exception as e:
            print(f"  ✗ {name:25} | Lỗi: {str(e)[:60]}")

    if not results_data:
        return

    # --- Bảng Xếp Hạng ---
    print("\nBẢNG XẾP HẠNG CÁC MÔ HÌNH (Cross-Validation):")
    results_df = pd.DataFrame(results_data).sort_values(by="F1-Macro", ascending=False)
    print(f"{'Rank':<5} {'Mô hình':<28} {'F1-Macro':<12} {'Accuracy':<10}")
    print("-" * 70)
    for idx, row in results_df.iterrows():
        rank = results_df.index.get_loc(idx) + 1
        print(f"{rank:<5} {row['Mô hình']:<28} {row['F1-Macro']:.4f} ±{row['Std']:.3f}   {row['Accuracy']:.4f}")

    top_3_models_df = results_df.iloc[0:3]
    top_3_model_names = top_3_models_df["Mô hình"].tolist()

    # --- GIAI ĐOẠN 3: Tối ưu hóa ---
    def objective_rf(trial):
        params = {
            'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 300),
            'model__max_depth': trial.suggest_int('model__max_depth', 5, 30),
        }
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestClassifier(random_state=42, class_weight='balanced'))])
        pipeline.set_params(**params)
        return np.mean(cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)['test_score'])

    def objective_gb(trial):
        params = {
            'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 300),
            'model__learning_rate': trial.suggest_float('model__learning_rate', 0.01, 0.3, log=True),
        }
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', GradientBoostingClassifier(random_state=42))])
        pipeline.set_params(**params)
        return np.mean(cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)['test_score'])

    def objective_svm(trial):
        params = {
            'model__C': trial.suggest_float('model__C', 0.1, 10, log=True),
            'model__kernel': trial.suggest_categorical('model__kernel', ['rbf', 'linear'])
        }
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', SVC(random_state=42, class_weight='balanced'))])
        pipeline.set_params(**params)
        return np.mean(cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)['test_score'])

    def objective_mlp(trial):
        params = {
            'model__hidden_layer_sizes': trial.suggest_categorical('model__hidden_layer_sizes', [(50,), (100,), (50, 50), (100, 50)]),
            'model__alpha': trial.suggest_float('model__alpha', 1e-5, 1e-2, log=True),
        }
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', MLPClassifier(random_state=42, max_iter=1000))])
        pipeline.set_params(**params)
        return np.mean(cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)['test_score'])

    def objective_ada(trial):
        params = {
            'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 300),
            'model__learning_rate': trial.suggest_float('model__learning_rate', 0.01, 1.0, log=True),
        }
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', AdaBoostClassifier(random_state=42, algorithm='SAMME'))])
        pipeline.set_params(**params)
        return np.mean(cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)['test_score'])

    def objective_lr(trial):
        params = {
            'model__C': trial.suggest_float('model__C', 0.01, 100, log=True),
            'model__solver': trial.suggest_categorical('model__solver', ['liblinear', 'saga']),
        }
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', LogisticRegression(random_state=42, max_iter=3000, class_weight='balanced'))])
        pipeline.set_params(**params)
        return np.mean(cross_validate(pipeline, X_train, y_train, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)['test_score'])

    tuning_objectives = {
        'Random Forest': objective_rf, 'Gradient Boosting': objective_gb,
        'Support Vector Machine': objective_svm, 'MLP Classifier': objective_mlp,
        'AdaBoost': objective_ada, 'Logistic Regression': objective_lr
    }

    tuned_results = {}

    print(f"\n--- Kết quả Tinh chỉnh Hyperparameters (Top 3) ---")
    for model_name in top_3_model_names:
        original_f1 = results_df[results_df["Mô hình"] == model_name]["F1-Macro"].values[0]

        if model_name in tuning_objectives:
            print(f"\n🔧 Đang tinh chỉnh {model_name}...")
            study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
            study.optimize(tuning_objectives[model_name], n_trials=30, show_progress_bar=True) # BẬT THANH PROGRESS

            best_params = {key.split('__')[1]: value for key, value in study.best_params.items()}
            best_cv_score = study.best_value
            print(f"  ✓ {model_name} (Tuned): F1 từ {original_f1:.4f} -> {best_cv_score:.4f}")
        else:
            best_params = {}
            best_cv_score = original_f1
            print(f"\n⚠ {model_name}: Không có kịch bản tối ưu (giữ nguyên F1 {original_f1:.4f})")

        tuned_model_instance = models.get(model_name)
        if best_params:
            try:
                tuned_model_instance.set_params(**best_params)
            except Exception: pass

        tuned_results[model_name] = {"cv_score": best_cv_score, "params": best_params, "model_instance": tuned_model_instance}

    # --- GIAI ĐOẠN 4: Đánh giá & Lưu ---
    print("\n--- Đánh giá Top 3 trên Test Set ---")
    final_test_scores = {}
    labels_order = ['None', 'Insomnia', 'Sleep Apnea']

    for model_name, results in tuned_results.items():
        final_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', results["model_instance"])])
        final_pipeline.fit(X_train, y_train)
        y_pred = final_pipeline.predict(X_test)
        test_metrics = calculate_metrics_summary(y_test, y_pred)
        final_test_scores[model_name] = test_metrics

        print(f"\n> Model: {model_name}")
        print(classification_report(y_test, y_pred, labels=labels_order, zero_division=0))

        # Tạo Confusion Matrix (Không in thông báo)
        try:
            cm = confusion_matrix(y_test, y_pred, labels=labels_order)
            fig, ax = plt.subplots(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels_order, yticklabels=labels_order, cbar_kws={'label': 'Số mẫu'}, ax=ax)
            ax.set_title(f'Ma trận Nhầm lẫn - {model_name}\nAccuracy: {test_metrics["Accuracy"]:.2%}', fontsize=14)
            ax.set_ylabel('Thực tế'); ax.set_xlabel('Dự đoán')
            plt.tight_layout()
            plt.savefig(f'final_confusion_matrix_{model_name.replace(" ", "_")}.png', dpi=150, bbox_inches='tight')
            plt.close()
        except Exception: pass

    # --- GIAI ĐOẠN 5: Chọn Model Tốt Nhất ---
    ultimate_winner_name = max(final_test_scores, key=lambda name: final_test_scores[name]['F1-Macro'])
    ultimate_winner_metrics = final_test_scores[ultimate_winner_name]
    winner_data = tuned_results[ultimate_winner_name]

    print(f"\n=== KẾT QUẢ CUỐI CÙNG ===")
    print(f" Model tốt nhất:       {ultimate_winner_name}")
    print(f" F1-Macro (Test):      {ultimate_winner_metrics['F1-Macro']:.4f}")
    print(f" Accuracy (Test):      {ultimate_winner_metrics['Accuracy']:.4f}")
    print(f" Precision-Macro:      {ultimate_winner_metrics['Precision-Macro']:.4f}")
    print(f" Recall-Macro:         {ultimate_winner_metrics['Recall-Macro']:.4f}")
    print(f" Params tối ưu:        {winner_data['params'] if winner_data['params'] else 'Mặc định'}")

    # Lưu model
    winner_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', winner_data["model_instance"])])
    winner_pipeline.fit(X_train, y_train)
    joblib.dump(winner_pipeline, 'sleep_disorder_model.joblib')

    # === GIAI ĐOẠN 6: Feature Importance (RF) ===
    try:
        rf_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100))
        ])
        rf_pipeline.fit(X_train, y_train)
        feature_names = rf_pipeline.named_steps['preprocessor'].get_feature_names_out()
        importances = rf_pipeline.named_steps['model'].feature_importances_
        forest_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)
        top_n = min(20, len(forest_importances))

        print(f"\nTop {top_n} Features quan trọng nhất:")
        print("-" * 50)
        for i, (feat, imp) in enumerate(forest_importances.head(top_n).items(), 1):
            print(f"{i:2}.    {feat:<35} {imp:.6f}")

        # Plot Feature Importance (Không in thông báo)
        plt.figure(figsize=(10, max(8, top_n * 0.5)))
        plt.barh(np.arange(top_n), forest_importances.head(top_n).values, color=plt.cm.viridis(np.linspace(0.3, 0.9, top_n)))
        plt.yticks(np.arange(top_n), forest_importances.head(top_n).index)
        plt.xlabel('Importance'); plt.title(f'Top {top_n} Features Importance')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.savefig('feature_importance_plot.png', dpi=150, bbox_inches='tight')
        plt.close()

    except Exception: pass

# --- CHẠY ---
if __name__ == "__main__":
    run_complete_ml_workflow_SleepDisorder()


Phân phối các lớp (Dataset):
  None            : 219 mẫu ( 58.6%)
  Sleep Apnea     :  78 mẫu ( 20.9%)
  Insomnia        :  77 mẫu ( 20.6%)

Phân phối lớp trong Train set:
  None            : 175 mẫu
  Sleep Apnea     :  62 mẫu
  Insomnia        :  62 mẫu

BẢNG XẾP HẠNG CÁC MÔ HÌNH (Cross-Validation):
Rank  Mô hình                      F1-Macro     Accuracy  
----------------------------------------------------------------------
1     MLP Classifier               0.8708 ±0.027   0.8962
2     Support Vector Machine       0.8649 ±0.035   0.8894
3     AdaBoost                     0.8648 ±0.020   0.8929
4     Logistic Regression          0.8606 ±0.033   0.8828
5     Gradient Boosting            0.8519 ±0.029   0.8796
6     Linear Discriminant          0.8454 ±0.045   0.8728
7     K-Nearest Neighbors          0.8421 ±0.032   0.8727
8     Random Forest                0.8271 ±0.042   0.8529
9     Decision Tree                0.8184 ±0.046   0.8428
10    Gaussian Naive Bayes         0.7413 ±0

  0%|          | 0/30 [00:00<?, ?it/s]

  ✓ MLP Classifier (Tuned): F1 từ 0.8708 -> 0.8730

🔧 Đang tinh chỉnh Support Vector Machine...


  0%|          | 0/30 [00:00<?, ?it/s]

  ✓ Support Vector Machine (Tuned): F1 từ 0.8649 -> 0.8714

🔧 Đang tinh chỉnh AdaBoost...


  0%|          | 0/30 [00:00<?, ?it/s]

  ✓ AdaBoost (Tuned): F1 từ 0.8648 -> 0.8698

--- Đánh giá Top 3 trên Test Set ---

> Model: MLP Classifier
              precision    recall  f1-score   support

        None       1.00      0.98      0.99        44
    Insomnia       0.93      0.87      0.90        15
 Sleep Apnea       0.83      0.94      0.88        16

    accuracy                           0.95        75
   macro avg       0.92      0.93      0.92        75
weighted avg       0.95      0.95      0.95        75


> Model: Support Vector Machine
              precision    recall  f1-score   support

        None       1.00      0.98      0.99        44
    Insomnia       0.81      0.87      0.84        15
 Sleep Apnea       0.88      0.88      0.88        16

    accuracy                           0.93        75
   macro avg       0.90      0.91      0.90        75
weighted avg       0.94      0.93      0.93        75


> Model: AdaBoost
              precision    recall  f1-score   support

        None       1.00